<h1><center>Laboratorio 8: Predicciones y Recomendaciones 🔮🪄 </center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2024</strong></center>

### **Cuerpo Docente:**

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### **Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados**

- Nombre de alumno 1: Diego Bartolucci
- Nombre de alumno 2: Pilar Nilo

### **Link de repositorio de GitHub:** [Repositorio](https://github.com/DiegoBarto01/MDS7202-pili-barto.git)

## Temas a tratar
- Series de Tiempo.
- Predicciones vía `Prophet`.
- Implementar un sistema de recomendación utilizando `surprise`.

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio
- Comprender qué es una serie de tiempo y su estructura.
- Identificar tendencias, estacionalidades e irregularidades.
- Armar un modelo predictivo para la serie.
- Conocer y aplicar sistemas de recomendación.
- Entender estructura y conocer casos de estudio.


In [ ]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = '/content/drive/MyDrive/Lab_8'
except:
    print('Ignorando conexión drive-colab')

In [2]:
# Librerias globales

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import plotly.express as px

# **1. Forecasting (30 puntos)**


## **Prophet**

Prophet es una herramienta open-source de Facebook utilizada para realizar predicciones en series de tiempo. Esta se basa en la descomposición aditiva, donde tendencias no lineales se ajustan junto a la estacionalidad.

En la ecuación de a continuación se puede ver una idea general de los elementos que construyen a un modelo aditivo como lo es Prophet.

\begin{equation}
y(t) = g(t) + s(t) + h(t) + e(t)
\end{equation}

Donde, $g(t)$ hace referencia a las tendencias, que corresponden a cambios graduales en largos periodos de tiempo. $s(t)$ corresponde a la estacionalidad, son cambios periodicos o cortos en el tiempo. $h(t)$ es el efecto que tienen las festividades sobre las predicciones, mientras que e(t) corresponde al error o ruido. Finalmente $y(t)$, es la predicción hecha por el modelo.

Prophet trabaja por defecto con Piece-Wise Lineal Model, este es un modelo de regresión lineal, en el cual se buscan distintas zonas en que la data presente patrones o tendencias lineales, de estas zonas obtiene su regresión y luego las "une" de manera de representar toda la región, como se puede ver en la ecuación siguiente.

\begin{equation}
y(x)=
    \begin{cases}
        η_1 + \beta_1(x-b_1), & b_1 < x  \leqslant b_2 \\
        η_2 + \beta_2(x-b_2), & b_2 < x  \leqslant b_3 \\
        η_3 + \beta_3(x-b_3), & b_3 < x  \leqslant b_3 \\
        ... \\
        η_k + \beta_{nb}(x-b_{nb-1}), & b_{n-1} < x  \leqslant b_{nb} \\
    \end{cases}
\end{equation}

Siendo $b_1$ el primer punto de quiebre en la serie y así hasta el punto $b_{nb}$ correspondiente al último punto de quiebre de la serie con una cantidad $nb$ de puntos.

Para mayor información de Prophet y como utilizarla, pueden ver su [documentación](https://facebook.github.io/prophet/docs/quick_start.html#python-api), donde hay pueden encontrar un pequeño tutorial de la librería.

## **La Factura de Homero**

<center>
<img src ="https://i.makeagif.com/media/7-30-2018/H_ZAY1.gif" width = 500 />

Homero Simpson ha trabajado en la Planta Nuclear de Springfield por más de 20 años como Inspector de Seguridad, pero recientemente el Sr. Burns le ha permitido trabajar desde casa. Aunque le encanta la flexibilidad que esto le otorga, también ha notado un aumento considerable en su consumo energético en el hogar. Con el uso constante de calefacción en invierno y aire acondicionado en verano, Homero está preocupado por cómo afectará esto a su presupuesto en los próximos meses, ya que planea pasar aún más tiempo en casa.

Afortunadamente, gracias a su puesto en la Planta de Energía Nuclear, Homero tiene acceso a los datos de consumo de energía de cada ciudadano en Springfield, por lo que accede a esta información y almacena el consumo de su hogar en un archivo llamado `energia_homero.csv`. Este archivo posee el consumo diario en el hogar de Homero Simpson desde el Junio del 2016 hasta Julio del 2020.

Con esta valiosa información, Homero espera poder predecir su consumo energético y tomar decisiones informadas para ajustar su presupuesto mensual.

Los datos son los siguientes:

* **date**: Fecha de medición.
* **Energy_kWh**: Consumo diario de energía eléctrica en el hogar de Homero (en kWh).

## 1.1 Series de Tiempo [0 puntos]

### Carga de los datos

En primer lugar, cargue los datos históricos del archivo `energia_homero.csv` al entorno de trabajo. Una vez cargados, asegúrense de transformar la columna `date` a formato `datetime`. Además, visualice el consumo de energía en la casa de Homero mediante la librería `plotly`.


**Respuesta**:

In [ ]:
#Se carga el archivo
cvs_path = path + '/energia_homero.csv'
df_energy = pd.read_csv(cvs_path)

#Se transforma la columna date a formato datetime
df_energy['date'] = pd.to_datetime(df_energy['date'])

#Se visualiza el consumo de energía en la casa de Homero
fig = px.line(df_energy, x='date', y='Energy_kWh', title='Consumo de energía en la casa de Homero')
fig.update_layout(xaxis_title='Fecha', yaxis_title='Consumo de energía (kWh)')
fig.show()

## 1.2 Controlando la Serie de Tiempo [6 puntos]

<center>
<img src ="https://i.pinimg.com/originals/66/77/88/667788e0b1f08ff1e1cfce11d303b203.gif" width = 500 />

Luego de leer mucho, Homero cree que necesita descomponer la serie de tiempo para poder controlar estos componentes de forma efectiva y lograr que la serie tenga propiedades de una serie estacionaria.

Usted, como buen ciudadano de Springfield, decide ayudar a Homero, el cual le pide que realice lo siguientes pasos:

0. Cree un conjunto de entrenamiento y uno de prueba (a proyectar). Para ello, considere como entrenamiento la información **hasta** el `2020-02-29` y como conjunto de prueba **desde** el `2020-03-01`. [0 puntos]
1. Cree un `Pipeline` que permita obtener una representación de la tendencia de la serie de tiempo, mediante un modelo Lineal. Nombre al pipeline `Pipeline_trend`. [1 punto]
2. Entrene el modelo lineal y luego obtenga predicciones tanto para el conjunto de entrenamiento como para el de prueba (intervalo completo de tiempo). Luego grafique las predicciones y calcule el `Mean Absolute Error` (MAE), tanto para el conjunto de prueba como para el de entrenamiento. [1 punto]
3. Grafique el error (Valor real - Valor predicho) para todo el intervalo de tiempo y responda ¿Hay un comportamiento estacional en la serie? [1 punto]
4.  Cree un `Pipeline` que permita modelar la estacionalidad de la serie temporal mediante términos de Fourier y la represente a través de una Regresión Lineal de la variable objetivo. Nombre a este pipeline `Pipeline_seasonal`. Determine el valor de Periodos `P` y el número de terminos de Fourier `n`. [1 punto]
5. Entrene el nuevo modelo y obtenga predicciones en el intervalo completo de tiempo. Luego grafique y calcule el `MAE` en el conjunto de prueba y en el de entrenamiento. [1 punto]
6. Vuelva a graficar el error y comente los resultados. [1 punto]

**Respuesta**

In [4]:
#0
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

#Se crean los conjuntos de entrenamiento y de test
train_df = df_energy[df_energy['date'] <= '2020-02-29']
test_df = df_energy[df_energy['date'] >= '2020-03-01']

In [5]:
#1
#Se crea un pipeline para obtener la tendencia de la serie de tiempo
pipeline_trend = Pipeline([
    ('scaler', StandardScaler()),
    ('linear_regression', LinearRegression())
])

In [ ]:
#2
from sklearn.metrics import mean_absolute_error
#Se entrena el modeo lineal para obtener predicciones para train_df y test_df.
pipeline_trend.fit(train_df[['date']], train_df['Energy_kWh'])
train_df['trend_pred'] = pipeline_trend.predict(train_df[['date']])
test_df['trend_pred'] = pipeline_trend.predict(test_df[['date']])

#Se grafican las predicciones y se calcula el Mean Absolute Error (MAE), tanto para test_df como para train_df
fig = px.line(train_df, x='date', y=['Energy_kWh', 'trend_pred'], title='Predicciones de la tendencia')
fig.update_layout(xaxis_title='Fecha', yaxis_title='Consumo de energía (kWh)')
fig.show()

fig = px.line(test_df, x='date', y=['Energy_kWh', 'trend_pred'], title='Predicciones de la tendencia')
fig.update_layout(xaxis_title='Fecha', yaxis_title='Consumo de energía (kWh)')
fig.show()

#Se calcula el MAE (Mean Absolute Error) para ambos conjuntos
mae_train = mean_absolute_error(train_df['Energy_kWh'],train_df['trend_pred'])
mae_test = mean_absolute_error(test_df['Energy_kWh'],test_df['trend_pred'])
print(f"MAE en el conjunto de entrenamiento: {mae_train}")
print(f"MAE en el conjunto de prueba: {mae_test}")

In [ ]:
#3
#Se calcula el error
predicted_values = np.concatenate([train_df['trend_pred'].values, test_df['trend_pred'].values])
error = df_energy['Energy_kWh'].values - predicted_values

#Se grafica el error para todo intervalo de tiempo
fig = px.line(df_energy, x='date', y=error, title='Error de la tendencia')
fig.update_layout(xaxis_title='Fecha', yaxis_title='Error')
fig.show()

En el gráfico se presenta cierta estacionalidad y periodicidad para el valor del error además de una gran variabilidad.

In [8]:
#4
#Se crea Pipeline mediante términos de Fourier y la represente a través de una Regresión Lineal
# Se inserta lo visto en clases para aplicar Fourier
from sklearn.base import BaseEstimator, TransformerMixin

class TimeTransformer(TransformerMixin, BaseEstimator):
    def __init__(self, n = 4, p = 365):

        self.init_date = None
        self.n = n
        self.p = p

    def fit(self, X, y=None):

        '''
        Realiza el fit sobre los datos --> guarda la fecha de inicio de entrenamiento.
        '''

        assert isinstance(X, pd.DataFrame), 'df must be a pandas DataFrame'
        assert 'date' in X.columns, 'date must be a column of DataFrame'

        self.init_date = X.date.iloc[0]

    def transform(self, X):

        '''
        Realiza el transform sobre el conjunto X, retornando los términos de fourier.
        '''

        X = X[['date']].copy()

        assert self.init_date is not None, 'transformer must be fitted first!'
        assert isinstance(X, pd.DataFrame), 'df must be a pandas Dataframe'
        assert 'date' in X.columns, 'date must be a column of DataFrame'

        final_date = X.index[-1] # last date of X
        final_date = X.date.iloc[-1] # last date of X

        # generate trend series
        count = (final_date - self.init_date).days + 1
        X['trend'] = [i for i in range(count)][-len(X):]

        # generate fourier terms
        fourier = self.get_fourier_terms(X['trend'], self.n, self.p)

        # concatenate trend + fourier
        X = pd.concat([X, fourier], axis = 1).drop(columns = 'date')

        return X

    def fit_transform(self, X, y=None):

        '''
        implementa el método fit y transform a la vez.
        '''

        X = X[['date']].copy()

        assert isinstance(X, pd.DataFrame), 'df must be a pandas Dataframe'
        assert 'date' in X.columns, 'date must be a column of DataFrame'

        self.trend = len(X)
        self.init_date = X.date.iloc[0]

        final_date = X.date.iloc[-1] # last date of X

        # generate trend series
        count = (final_date - self.init_date).days + 1
        X['trend'] = [i for i in range(count)][-len(X):]

        # generate fourier terms
        fourier = self.get_fourier_terms(X['trend'], self.n, self.p)

        # concatenate trend + fourier
        X = pd.concat([X, fourier], axis = 1).drop(columns = 'date')

        return X

    def get_fourier_terms(self, serie, n, p):

        '''
        Obtiene los términos de fourier de una serie
        '''

        terms = []
        for i in range(1, n + 1):
            sine_term = np.sin(2 * np.pi * i * serie / p)
            cosine_term = np.cos(2 * np.pi * i * serie / p)
            terms.extend([sine_term, cosine_term])

        fourier = np.column_stack(terms)
        fourier = pd.DataFrame(fourier, columns = [f'fourier_{i}' for i in range(n * 2)], index = serie.index) # este paso es opcional

        return fourier

pipeline_seasonal = Pipeline([
    ('time_transformer', TimeTransformer(n = 4, p = 365)),
    ('scaler', StandardScaler()),
    ('linear_regression', LinearRegression())
])

Se utilizará una periodicidad de 365 ya que en un año no bisiesto se presentan esta cantidad de días y se usará una cantidad de terminos de 4 ya que con este valor se presentaron los mejores resultados.

In [ ]:
#5
#Se entrena el nuevo modelo
pipeline_seasonal.fit(train_df[['date']], train_df['Energy_kWh'])
train_df['seasonal_pred'] = pipeline_seasonal.predict(train_df[['date']])
test_df['seasonal_pred'] = pipeline_seasonal.predict(test_df[['date']])

#Se grafican las predicciones y se calcula el Mean Absolute Error (MAE), tanto para test_df como para train_df
fig = px.line(train_df, x='date', y=['Energy_kWh', 'seasonal_pred'], title='Predicciones de la tendencia')
fig.update_layout(xaxis_title='Fecha', yaxis_title='Consumo de energía (kWh)')
fig.show()

fig = px.line(test_df, x='date', y=['Energy_kWh', 'seasonal_pred'], title='Predicciones de la tendencia')
fig.update_layout(xaxis_title='Fecha', yaxis_title='Consumo de energía (kWh)')
fig.show()

#Se calcula el MAE (Mean Absolute Error) para ambos conjuntos
mae_train = mean_absolute_error(train_df['Energy_kWh'],train_df['seasonal_pred'])
mae_test = mean_absolute_error(test_df['Energy_kWh'],test_df['seasonal_pred'])
print(f"MAE en el conjunto de entrenamiento: {mae_train}")
print(f"MAE en el conjunto de prueba: {mae_test}")

In [ ]:
#6
#Se calcula el error
predicted_values_seasonal = np.concatenate([train_df['seasonal_pred'].values, test_df['seasonal_pred'].values])
error_seasonal = df_energy['Energy_kWh'].values - predicted_values_seasonal

#Se grafica el error para todo intervalo de tiempo
fig = px.line(df_energy, x='date', y=error_seasonal, title='Error de la tendencia')
fig.update_layout(xaxis_title='Fecha', yaxis_title='Error')
fig.show()

Se puede observar que el error disminuyó, infiriendo así que el modelo representa mejor los datos buscados, además que el comportamiento que presenta el error tiene más semenjanzas a una onda estacionaria donde se presenta una mayor variabilidad durante los meses de invierno.

## 1.3 Un Consejo para Homero [6 puntos]

<center>
<img src ="https://media.tenor.com/sjbvGMLZiDkAAAAM/the-simpsons-homner-simpson.gif" width = 400 />

Homero al observar los resultados obtenidos, se mostró conforme y decidió no seguir averiguando al respecto. Usted, como buen amigo y utilizando sus recientes conocimientos en Forecast, le sugiere a Homero utilizar **Prophet**, una herramienta de modelado de series de tiempo que incluye descomposición de tendencias y estacionalidad, así como la capacidad de incorporar regresores adicionales. Usted le explica a Homero que Prophet es especialmente útil para realizar pronósticos cuando hay patrones estacionales y que podría mejorar los resultados que ha obtenido hasta ahora.

Homero, algo agotado debido al exhaustivo estudio que hizo para entender como funciona esta herramienta, le solicita ayuda para implementar Prophet y evaluar si puede obtener mejores resultados en sus predicciones de consumo energético.

Dado esto, Homero le pide que realice los siguientes pasos:

1. Instale y cargue Prophet a su entorno de trabajo. Luego renombre la variable temporal `date` por `ds` y la variable objetivo `Energy_kWh` por `y`, en los conjuntos de entrenamiento y de prueba. [1 punto]
2. Cree y ajuste el modelo Prophet con sus datos de entrenamiento. Luego realice prepare el dataset de predicciones con su set de prueba y realice las predicciones. `Nota:` Le puede ser útil este [Enlace](https://facebook.github.io/prophet/docs/quick_start.html#python-api) [2 puntos]
3. Grafique las predicciones y las componentes del modelo Prophet. `Nota`: Utilice los métodos `.plot()` y `.plot_components()` de Prophet. [1 punto]
4. Calcule el `MAE` de la predicción en el conjunto de entrenamiento y de prueba. [1 punto]
5. Grafique el error (Valor real - Valor predicho) en todo el intervalo de tiempo. Analice sus resultados y responda: ¿Hay un comportamiento estacional en la serie? [1 punto]

In [11]:
# librerias extras
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly

**Respuesta:**

In [12]:
#1
#Se adaptan los conjuntos de entrenamiento y test
df_energy_prophet = df_energy.rename(columns={'date': 'ds', 'Energy_kWh': 'y'})
train_df_prophet = df_energy_prophet[df_energy_prophet['ds'] <= '2020-02-29']
test_df_prophet = df_energy_prophet[df_energy_prophet['ds'] >= '2020-03-01']

In [ ]:
#2
#Se crea y ajusta el modelo prophet con los datos de entrenamiento
model = Prophet()
model.fit(train_df_prophet)

In [14]:
#Se prepara el dataset de predicciones con su set de prueba y realice las predicciones.
future = pd.DataFrame({'ds': test_df_prophet['ds']})
forecast = model.predict(future)

In [ ]:
#3
#Se grafican las predicciones y componentes del modelo prophet
fig_predict = model.plot(forecast)
ax_predict = fig_predict.gca()
ax_predict.set_title("Predicción del consumo de energía")
ax_predict.set_xlabel("Fecha")
ax_predict.set_ylabel("Consumo de Energía (kWh)")

fig_components = model.plot_components(forecast)
for ax in fig_components.axes:
    ax.set_xlabel("Fecha")
    if ax.get_ylabel() == "trend":
        ax.set_ylabel("Tendencia")
    elif ax.get_ylabel() == "yearly":
        ax.set_ylabel("Estacionalidad Anual")
    elif ax.get_ylabel() == "weekly":
        ax.set_ylabel("Estacionalidad Semanal")

In [ ]:
#4
#Calcule el MAE de la predicción en el conjunto de entrenamiento y de prueba.
mae_train_prophet = mean_absolute_error(train_df_prophet['y'], model.predict(train_df_prophet)['yhat'])
mae_test_prophet = mean_absolute_error(test_df_prophet['y'], forecast['yhat'])
print(f"MAE en el conjunto de entrenamiento: {mae_train_prophet}")
print(f"MAE en el conjunto de prueba: {mae_test_prophet}")

In [ ]:
#5
#Se grafica el error (Valor real - Valor predicho) en todo el intervalo de tiempo.
error_prophet = df_energy_prophet['y'] - model.predict(df_energy_prophet)['yhat']
fig = px.line(df_energy_prophet, x='ds', y=error_prophet, title='Error de Prophet')
fig.update_layout(xaxis_title='Fecha', yaxis_title='Error')
fig.show()

Se puede observar que se mantienen la estacionalidad de la serie durante los meses de invierno, pero presentandose una menor varianza en los datos.

## 1.4 Incluyendo Regresores [5 puntos]

<center>
<img src ="https://i.redd.it/64hwjftunjjd1.gif" width = 400 />

Con los nuevos resultados obtenidos del modelo Prophet, usted le aconseja a Homero que puede mejorar sus predicciones al incluir regresores en el modelo, es decir, variables adicionales que pueden influir en la variable objetivo, en este caso, el consumo energético. Homero encuentra que es una buena idea y va en busca de ayuda donde el Profesor Frink, un destacado Científico de Springfield. Para suerte de Homero, Frink tenía un estudio reciente que registra las condiciones climáticas en Springfield, en el mismo intervalo de tiempo en que Homero disponía de los datos de consumo eléctrico. Con esta nueva información, usted le recomienda a Homero que podrían utilizar esta información meteorológica proporcionada como variables exógenas al modelo.

Sin embargo, al no comprender bien la importancia de los regresores, Homero decide confiar en sus capacidades y le pide que, por favor, incluya estas variables en el modelo para evaluar su impacto en las predicciones.

Para ello, usted debe:
0. Seleccionar las variables exógenas a utilizar del archivo `datos_frink.csv`. Añada estas variables a sus conjuntos de entrenamiento y prueba mediante `merge`. [0 puntos]
1. Cree el modelo Prophet e incluya los regresores (variables exógenas) al modelo. Luego realice el entrenamiento. **Hint**: Utilice el método `.add_regressor()`. Mas información en el siguiente [Enlace](https://facebook.github.io/prophet/docs/seasonality,_holiday_effects,_and_regressors.html#additional-regressors). [2 puntos]
2. Grafique las predicciones y las componentes del modelo Prophet. `Nota`: Utilice los métodos `.plot()` y `.plot_components()` de Prophet. [1 punto]
3. Calcule el `MAE` de la predicción en el conjunto de entrenamiento y de prueba. Analice sus resultados. [1 punto]
4. Grafique el error (Valor real - Valor predicho) en el conjunto de entrenamiento y de prueba. ¿Hay un comportamiento estacional en la serie? ¿Influyen estas variables en el comportamiento del consumo energético en el hogar de Homero? [1 punto]

Los datos del dataset de Frink son los siguientes:

* **date**: Fecha de medición.
* **Temp_max**: Temperatura máxima registrada durante el día (en °F).
* **Temp_avg**: Temperatura promedio registrada durante el día (en °F).
* **Temp_min**: Temperatura mínima registrada durante el día (en °F).
* **Dew_max**: Punto de rocío máximo registrado durante el día (en °F).
* **Dew_avg**: Punto de rocío promedio registrado durante el día (en °F).
* **Dew_min**: Punto de rocío mínimo registrado durante el día (en °F).
* **Hum_max**: Humedad máxima registrada durante el día (en porcentaje).
* **Hum_avg**: Humedad promedio registrada durante el día (en porcentaje).
* **Hum_min**: Humedad mínima registrada durante el día (en porcentaje).
* **Wind_max**: Velocidad máxima del viento registrada durante el día (en millas por hora, mph).
* **Wind_avg**: Velocidad promedio del viento registrada durante el día (en millas por hora, mph).
* **Wind_min**: Velocidad mínima del viento registrada durante el día (en millas por hora, mph).
* **Press_max**: Presión atmosférica máxima registrada durante el día (en pulgadas de mercurio, Hg).
* **Press_avg**: Presión atmosférica promedio registrada durante el día (en pulgadas de mercurio, Hg).
* **Press_min**: Presión atmosférica mínima registrada durante el día (en pulgadas de mercurio, Hg).

**Respuesta:**

In [18]:
#0
#Se cargan los datos del dataset de Frink
cvs_frink = path + '/datos_frink.csv'
df_frink = pd.read_csv(cvs_frink)

#Se transforma la columna date a formato datetime
df_frink['date'] = pd.to_datetime(df_frink['date'])

#Se unen a través de merge a los datos de prueba y entrenamiento
df_energy_frink = df_energy.merge(df_frink, on='date', how='left')

#Se ajustan los datos para prophet
df_energy_frink_prophet = df_energy_frink.rename(columns={'date': 'ds', 'Energy_kWh': 'y'})
train_df_frink_prophet = df_energy_frink_prophet[df_energy_frink_prophet['ds'] <= '2020-02-29']
test_df_frink_prophet = df_energy_frink_prophet[df_energy_frink_prophet['ds'] >= '2020-03-01']

In [ ]:
#1
#Para seleccionar las variables exogenas se utilizará la correlación que presenten las variables
correlacion = df_energy_frink_prophet.corr()
correlacion.style.background_gradient(cmap='coolwarm')

In [ ]:
#Se crea el modelo Prophet e incluya los regresores al modelo. Hint: Utilice el método .add_regressor().
model_frink = Prophet()
var_exogenas = ['Temp_min','Temp_avg','Temp_max','Dew_min','Dew_avg','Dew_max']
for var in var_exogenas:
  model_frink.add_regressor(var)
model_frink.fit(train_df_frink_prophet)

#Se realiza el entrenamiento.
future_frink = test_df_frink_prophet[['ds', 'Temp_min','Temp_avg','Temp_max','Dew_min','Dew_avg','Dew_max']]
forecast_frink = model_frink.predict(future_frink)

In [ ]:
#2
#Se grafican las predicciones y componentes del modelo
fig_predict_frink = model_frink.plot(forecast_frink)
ax_predict_frink = fig_predict_frink.gca()
ax_predict_frink.set_title("Predicción del consumo de energía")
ax_predict_frink.set_xlabel("Fecha")
ax_predict_frink.set_ylabel("Consumo de Energía (kWh)")

fig_components_frink = model_frink.plot_components(forecast_frink)
for ax in fig_components_frink.axes:
    ax.set_xlabel("Fecha")
    if ax.get_ylabel() == "trend":
      ax.set_ylabel("Tendencia")
    elif ax.get_ylabel() == "yearly":
        ax.set_ylabel("Estacionalidad Anual")
    elif ax.get_ylabel() == "weekly":
        ax.set_ylabel("Estacionalidad Semanal")
    elif ax.get_ylabel() == "extra_regressors_additive":
        ax.set_ylabel("Efecto de Regresores Adicionales")

In [ ]:
#3
#Se calcula el MAE de la predicción en el conjunto de entrenamiento y de prueba.
mae_train_frink = mean_absolute_error(train_df_frink_prophet['y'], model_frink.predict(train_df_frink_prophet)['yhat'])
mae_test_frink = mean_absolute_error(test_df_frink_prophet['y'], forecast_frink['yhat'])
print(f"MAE en el conjunto de entrenamiento: {mae_train_frink}")
print(f"MAE en el conjunto de prueba: {mae_test_frink}")

*Introduzca su respuesta aquí*

In [ ]:
#4
#Se separan los valores de train y test por una etiqueta nueva en el df base
fecha_separacion = pd.to_datetime('2020-02-29')
df_energy_frink_prophet_traintest = df_energy_frink_prophet.copy()
df_energy_frink_prophet_traintest['train_test'] = df_energy_frink_prophet_traintest['ds'].apply(lambda x: 'train' if x <= fecha_separacion else 'test')
#Se grafica el error (Valor real - Valor predicho) en el conjunto de entrenamiento y de prueba.
error_frink_prophet = df_energy_frink_prophet_traintest['y'] - model_frink.predict(df_energy_frink_prophet_traintest)['yhat']
fig = px.line(df_energy_frink_prophet_traintest, x='ds', y=error_frink_prophet, color='train_test', title='Error de Prophet')
fig.update_layout(xaxis_title='Fecha', yaxis_title='Error')
fig.show()

Se puede observar estacionalidad en la serie de igual manera que en los gráficos anteriores, pero se ve una mejora en la varianza presente en los datos. Esto se asume que se debe a los regresores implementados.

## 1.5 Aplicando Lags [7 puntos]

<center>
<img src ="https://i.gifer.com/J45h.gif" width = 400 />

Homero, satisfecho con los resultados obtenidos, cree que aún puede mejorar el rendimiento del modelo incorporando lags de las variables exógenas. Se le ocurre que, al incluir estos lags, podría identificar cómo los valores pasados de las variables influyen en la evolución de la variable objetivo. Con esta idea en mente, Homero solicita su ayuda para encontrar y aplicar los lags adecuados a las variables exógenas de manera efectiva.

Para ello, a usted se le pide que:

1. Calcule la `Autocorrelación Parcial` de las variables exógenas `avg` que usted haya considerado. Por ejemplo: `Temp_avg, Dew_avg, etc`. Luego determine la cantidad de rezagos (lags) que se va a aplicar a cada una de las variables. Justifique. [1 punto]

2. Aplique los lags determinados en el paso anterior e incluyalos como variables en los conjuntos train y test. [1 punto]

3. Impute los valores nulos de los nuevos atributos lags mediante la media de cada variable. [1 punto]

4. Cree y entrene el modelo Prophet e incluya las variables exógenas y adicionalmente sus respectivos lags al modelo. [1 punto]

5. Grafique las predicciones y las componentes del modelo Prophet. `Nota`: Utilice los métodos `.plot()` y `.plot_components()` de Prophet. [1 punto]
6. Calcule el `MAE` de la predicción en el conjunto de prueba y en el de entrenamiento. Analice sus resultados. [1 punto]
7. Grafique el error (Valor real - Valor predicho) en el conjunto de entrenamiento y en el de prueba. ¿Hay un comportamiento estacional en la serie? ¿Como afecta la inclusión de los lags en el desempeño del modelo? [1 punto]


**Respuesta:**

In [ ]:
#1
#Se calcula la autocorrelación parcial de cada variable avg elegida
from statsmodels.graphics.tsaplots import plot_pacf
for var in ['Temp_avg', 'Dew_avg']:
  fig, ax = plt.subplots(figsize=(12, 6))
  plot_pacf(df_energy_frink_prophet[var], lags=90, alpha = 0.05, ax = ax)
  plt.title('Partial Autocorrelation Function ' + var)
  plt.show()

*Inserte su respuesta aquí*

In [25]:
#2
#Se aplican los lags determinados en el paso anterior y se incluyen como variables en los conjuntos train y test.
df_energy_frink_prophet_lag = df_energy_frink_prophet.copy()
df_energy_frink_prophet_lag['Temp_avg_lag'] = df_energy_frink_prophet_lag['Temp_avg'].shift(4)
df_energy_frink_prophet_lag['Dew_avg_lag'] = df_energy_frink_prophet_lag['Dew_avg'].shift(4)

In [26]:
#3
#Se imputan los valores nulos de los nuevos atributos lags mediante la media de cada variable.
df_energy_frink_prophet_lag['Temp_avg_lag'] = df_energy_frink_prophet_lag['Temp_avg_lag'].fillna(df_energy_frink_prophet_lag['Temp_avg_lag'].mean())
df_energy_frink_prophet_lag['Dew_avg_lag'] = df_energy_frink_prophet_lag['Dew_avg_lag'].fillna(df_energy_frink_prophet_lag['Dew_avg_lag'].mean())

In [ ]:
#4
#Se crean los conjuntos de entrenamiento y testeo
train_df_frink_prophet_lag = df_energy_frink_prophet_lag[df_energy_frink_prophet_lag['ds'] <= '2020-02-29']
test_df_frink_prophet_lag = df_energy_frink_prophet_lag[df_energy_frink_prophet_lag['ds'] >= '2020-03-01']

#Se crea el modelo prophet y se entrena con los regresores más los lags
model_frink_lag = Prophet()
for var in var_exogenas:
  model_frink_lag.add_regressor(var)
#Se agregan los lags
model_frink_lag.add_regressor('Temp_avg_lag')
model_frink_lag.add_regressor('Dew_avg_lag')
model_frink_lag.fit(train_df_frink_prophet_lag)

#Se realiza el entrenamiento.
future_frink_lag = test_df_frink_prophet_lag[['ds', 'Temp_min','Temp_avg','Temp_max','Dew_min','Dew_avg','Dew_max','Temp_avg_lag','Dew_avg_lag']]
forecast_frink_lag = model_frink_lag.predict(future_frink_lag)

In [ ]:
#5
#Se grafican las predicciones y componentes del modelo
fig_predict_frink_lag = model_frink_lag.plot(forecast_frink_lag)
ax_predict_frink_lag = fig_predict_frink_lag.gca()
ax_predict_frink_lag.set_title("Predicción del consumo de energía")
ax_predict_frink_lag.set_xlabel("Fecha")
ax_predict_frink_lag.set_ylabel("Consumo de Energía (kWh)")

fig_components_frink_lag = model_frink_lag.plot_components(forecast_frink_lag)
for ax in fig_components_frink_lag.axes:
    ax.set_xlabel("Fecha")
    if ax.get_ylabel() == "trend":
      ax.set_ylabel("Tendencia")
    elif ax.get_ylabel() == "yearly":
        ax.set_ylabel("Estacionalidad Anual")
    elif ax.get_ylabel() == "weekly":
        ax.set_ylabel("Estacionalidad Semanal")
    elif ax.get_ylabel() == "extra_regressors_additive":
        ax.set_ylabel("Efecto de Regresores Adicionales")

In [ ]:
#6
#Se calcula el MAE de la predicción en el conjunto de entrenamiento y de prueba.
mae_train_frink_lag = mean_absolute_error(train_df_frink_prophet_lag['y'], model_frink_lag.predict(train_df_frink_prophet_lag)['yhat'])
mae_test_frink_lag = mean_absolute_error(test_df_frink_prophet_lag['y'], forecast_frink_lag['yhat'])
print(f"MAE en el conjunto de entrenamiento: {mae_train_frink_lag}")
print(f"MAE en el conjunto de prueba: {mae_test_frink_lag}")

In [ ]:
#7
#Se separan los valores de train y test por una etiqueta nueva en el df base
fecha_separacion = pd.to_datetime('2020-02-29')
df_energy_frink_prophet_lag = pd.concat([train_df_frink_prophet_lag, test_df_frink_prophet_lag])
df_energy_frink_prophet_lag['train_test'] = df_energy_frink_prophet_lag['ds'].apply(lambda x: 'train' if x <= fecha_separacion else 'test')
#Se grafica el error (Valor real - Valor predicho) en el conjunto de entrenamiento y de prueba.
error_frink_prophet_lag = df_energy_frink_prophet_lag['y'] - model_frink_lag.predict(df_energy_frink_prophet_lag)['yhat']
fig = px.line(df_energy_frink_prophet_lag, x='ds', y=error_frink_prophet_lag, color='train_test', title='Error de Prophet')
fig.update_layout(xaxis_title='Fecha', yaxis_title='Error')
fig.show()

Se puede observar que hubo una leve mejora al utilizar lags en el conjunto de entrenamiento pero un leve empeoramiento en el conjunto de prueba, manteniendo la estacionalidad general del error presentado. En aspectos generales el uso de variables con lag no afecta en gran medida el comportamiento del modelo.

## 1.6 Optimizando Prophet [6 puntos]

<center>
<img src ="https://i.gifer.com/BKlg.gif" width = 500 />

Homero, contento con sus resultados, decide compartir su trabajo en LinkedIn. Sin embargo, justo antes de realizar la publicación, su hija Lisa Simpson le comenta que su modelo podría mejorar aún más mediante la optimización de hiperparámetros. Aunque Homero no entiende muy bien a qué se refiere Lisa, decide confiar en su conocimiento y le pide ayuda para implementar la optimización.

Lisa le prepara una función llamada optimize_prophet, que recibe como entrada el registro histórico, los datos a predecir (futuros), el nombre de las variables exógenas y una grilla de hiperparámetros que se desea modificar.

Después de varios intentos, Homero no logra utilizar la función de Lisa y, desesperado por publicar sus resultados en internet, se acerca a usted en busca de ayuda.

Para ello, usted decide realizar los siguientes pasos:

1. Crear una grilla donde se modifiquen los siguientes hiperparámetros: `changepoint_prior_scale`, `seasonality_prior_scale`, `seasonality_mode`, `changepoint_range` y encuentre los hiperparámetros que optimicen el `MAE`. `Nota:` Utilice la función `optimize_prophet()` dada en el enunciado. [1 punto]
2. Crear y entrenar un modelo Prophet utilizando los mejores parámetros encontrados, incluyendo regresores (variables exógenas). [2 puntos]
3. Grafique las predicciones y las componentes del modelo Prophet. `Nota`: Utilice los métodos `.plot()` y `.plot_components()` de Prophet. [1 punto]
4. Calcule el `MAE` de la predicción en el conjunto de entrenamiento y en el de prueba. [1 punto]
5. Grafique el error (Valor real - Valor predicho) en ambos conjuntos. Analice sus resultados. [1 punto]


In [31]:
#NO MODIFICAR ESTA CELDA

import itertools
import numpy as np
import pandas as pd
from tqdm import tqdm
import logging

# Silenciar el logging de Prophet
logging.getLogger('prophet').setLevel(logging.ERROR)
logging.getLogger('cmdstanpy').setLevel(logging.ERROR)


def optimize_prophet(df, df_to_pred, features, param_grid ):
  '''
  Función que recibe como entrada:
  -df: data entrenamiento.
  -df_to_pred: data a predecir (test).
  -features: lista de variables exógenas.
  -param_grid: grilla de hiperparámetros.

  Retorna:
  -tuning_results: Dataframe con resultados de la optimización.
  -best_params: Diccionario de mejores parámetros encontrados.
  '''

  # Generate all combinations of parameters
  all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
  MAEs = []  # Store the MAEs for each params here

  # Use tqdm to add a progress bar to the parameter tuning loop
  for params in tqdm(all_params, desc="Tuning Prophet parameters"):
      m = Prophet(**params)  # Fit model with given params
      for feature in features:
          m.add_regressor(feature)
      m.fit(df)

      # Prepare future dataframe
      df_prop = m.make_future_dataframe(periods=len(df_to_pred))
      df_feat = pd.concat([df[features], df_to_pred[features]]).reset_index(drop=True)
      df_prop[features] = df_feat[features]

      # Make predictions
      forecast = m.predict(df_prop)
      forecast_pred = forecast[forecast['ds'].isin(df_to_pred['ds'])].reset_index(drop=True)

      # Calculate MAE for the predictions
      mae_pred = mean_absolute_error(df_to_pred['y'], forecast_pred['yhat'])
      MAEs.append(mae_pred)

  # Find the best parameters
  tuning_results = pd.DataFrame(all_params)
  tuning_results['MAEs'] = MAEs
  tuning_results = tuning_results.sort_values(by='MAEs', ascending=True)
  best_params = all_params[np.argmin(MAEs)]
  return tuning_results, best_params

**Respuesta:**

In [ ]:
#1
#Creamos los conjuntos de entrenamiento y de testeo para esta sección
train_df_frink_prophet_opt = train_df_frink_prophet.copy()
test_df_frink_prophet_opt = test_df_frink_prophet.copy()

#Buscamos los mejores hiperparámetros para el modelo
grilla = {'changepoint_prior_scale' : [1, 0.1, 0.001], 'seasonality_prior_scale': [0.1, 1, 10],
          'seasonality_mode' :['additive', 'multiplicative'], 'changepoint_range': [0.8, 0.85, 0.9]}
tuning_results, best_params = optimize_prophet(train_df_frink_prophet_opt,test_df_frink_prophet_opt,var_exogenas,grilla)
print(tuning_results)
print(best_params)

In [34]:
#2
#Se crea y entrena el modelo prophet con los mejores parámetros encontrados
model_frink_opt = Prophet(**best_params)
for var in var_exogenas:
  model_frink_opt.add_regressor(var)

model_frink_opt.fit(train_df_frink_prophet_opt)
future_frink_opt = test_df_frink_prophet_opt[['ds', 'Temp_min','Temp_avg','Temp_max','Dew_min','Dew_avg','Dew_max']]
forecast_frink_opt = model_frink_opt.predict(future_frink_opt)

In [ ]:
#3
#Se grafican las predicciones y componentes del modelo
fig_predict_frink_opt = model_frink_opt.plot(forecast_frink_opt)
ax_predict_frink_opt = fig_predict_frink_opt.gca()
ax_predict_frink_opt.set_title("Predicción del consumo de energía")
ax_predict_frink_opt.set_xlabel("Fecha")
ax_predict_frink_opt.set_ylabel("Consumo de Energía (kWh)")

fig_components_frink_opt = model_frink_opt.plot_components(forecast_frink_opt)
for ax in fig_components_frink_opt.axes:
    ax.set_xlabel("Fecha")
    if ax.get_ylabel() == "trend":
      ax.set_ylabel("Tendencia")
    elif ax.get_ylabel() == "yearly":
        ax.set_ylabel("Estacionalidad Anual")
    elif ax.get_ylabel() == "weekly":
        ax.set_ylabel("Estacionalidad Semanal")
    elif ax.get_ylabel() == "extra_regressors_additive":
        ax.set_ylabel("Efecto de Regresores Adicionales")

In [ ]:
#4
#Se calcula el MAE de la predicción en el conjunto de entrenamiento y de prueba.
mae_train_frink_opt = mean_absolute_error(train_df_frink_prophet_opt['y'], model_frink_opt.predict(train_df_frink_prophet_opt)['yhat'])
mae_test_frink_opt = mean_absolute_error(test_df_frink_prophet_opt['y'], forecast_frink_opt['yhat'])
print(f"MAE en el conjunto de entrenamiento: {mae_train_frink_opt}")
print(f"MAE en el conjunto de prueba: {mae_test_frink_opt}")

In [ ]:
#5
#Se separan los valores de train y test por una etiqueta nueva en el df base
fecha_separacion = pd.to_datetime('2020-02-29')
df_energy_frink_prophet_opt = pd.concat([train_df_frink_prophet_opt, test_df_frink_prophet_opt])
df_energy_frink_prophet_opt['train_test'] = df_energy_frink_prophet_opt['ds'].apply(lambda x: 'train' if x <= fecha_separacion else 'test')
#Se grafica el error (Valor real - Valor predicho) en el conjunto de entrenamiento y de prueba.
error_frink_prophet_opt = df_energy_frink_prophet_opt['y'] - model_frink_opt.predict(df_energy_frink_prophet_opt)['yhat']
fig = px.line(df_energy_frink_prophet_opt, x='ds', y=error_frink_prophet_opt, color='train_test', title='Error de Prophet')
fig.update_layout(xaxis_title='Fecha', yaxis_title='Error')
fig.show()

Como se puede observar en el gráfico y en los valores del MAE, utilizando los mejores parámetros se obtuvieron mejores resultados, disminuyendo el MAE tanto para el conjunto de entrenamiento como para el de testeo, además se puede observar un gráfico más homogéneo que los anteriores.

# **2. Sistemas de Recomendación (30 puntos)**

## Homero contra la Prohibición

<center>
<img src ="https://i.ytimg.com/vi/rMSXXuhZjYY/maxresdefault.jpg" width = 500 />

En Springfield, el caos estalló cuando las autoridades prohibieron el alcohol. Pero como siempre, donde otros ven problemas, Homero Simpson ve oportunidades. Decidido a mantener las fiestas vivas, se convirtió en el Barón de la Cerveza en un abrir y cerrar de ojos. Con la ayuda de su buen amigo Moe el cantinero, Homero obtuvo un valioso dataset llamado `cervezas.csv`, que contiene valoraciones de los ciudadanos de Springfield sobre distintas cervezas. Homero quiere asegurarse de ofrecer las cervezas más apreciadas para mantener a su clientela oculta y feliz, mientras eludía la mirada del estricto Elio Pez. Por ende, Homero lo contacta para solicitar de su ayuda en esta nueva labor.

El dataset contiene los siguientes atributos:

  * userId: Identificador de los ciudadanos/usuarios
  * beerId: Identificador único de cada cerveza
  * rating: Evaluación otorgada por el ciudadano a la cerveza, en un rango de 1.0 a 5.0.
  * beerName: Nombre de la cerveza
  * beerStyle: Tipo de Cerveza.



In [ ]:
!pip install surprise

In [ ]:
# Librerias extras
from sklearn.preprocessing import LabelEncoder

from surprise import accuracy
from surprise.reader import Reader
from surprise.dataset import Dataset
from surprise.prediction_algorithms.knns import KNNBasic
from surprise.prediction_algorithms.matrix_factorization import SVD
from surprise.accuracy import mae
from surprise import NMF
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import pandas as pd
from copy import deepcopy

## Carga de los datos [0 puntos]

Cargue el dataset `cervezas.csv` y realice una breve exploración de los datos. ¿Hay valores nulos? ¿Cuantos cervezas se estan evaluando? ¿Cuantos ciudadanos participaron de esta evaluación?

In [ ]:
def exploratory_data_analysis(dataframe):
  #1. dimension df
  print(f"1. La dimension del dataframe es: {dataframe.shape} \n")
  #2. nombres de las columnas del df
  print(f"2. Las columnas que conforman el dataframe son: \n {dataframe.columns} \n")
  #3. mostrar los primeros 5 elementos, 5 ultimos elementos y 5 elementos aleatoreos
  print("3.a Los 5 primeros elementos del dataframe son:")
  display(dataframe.head(5))
  print("3.b Los 5 últimos elementos del dataframe son:")
  display(dataframe.tail(5))
  print("3.c 5 elementos al azar del dataframe son:")
  display(dataframe.sample(5, random_state=10))
  #4. descripcion rapida de las variables numericas del dataframe
  print(f"4. Descripción de las variables numéricas del dataframe:")
  display(dataframe.describe())
  #5. mensaje con la cantidad de valores nulos
  print(f"5. La cantidad de valores nulos en el dataframe son: \n {dataframe.isna().sum()} \n")
  #6. mensaje que detalle la cantidad de valores unicos
  print(f"6. La cantidad de valores unicos en el dataframe son: \n {dataframe.nunique()} \n")
  #7. mensaje con el total de filas que contengan nombres duplicados
  print(f"7. Total de filas que contienen userId duplicados: {dataframe.duplicated(subset=['userId']).sum()} \n")
  #8. Mostrar las filas que contengan nombres sean duplicados
  print(f"8. Filas que contienen userId duplicados:")
  display(dataframe[dataframe.duplicated(subset=['userId'])])

In [ ]:
#cargamos el dataset
cvs_cervezas = path + '/cervezas.csv'
df_cervezas = pd.read_csv(cvs_cervezas)

In [ ]:
#exploramos los datos
exploratory_data_analysis(df_cervezas)

*analisis exploratorio del dataset*

Mediante la exploración realizada, no hay valores nulos en el dataset, la cantidad de cervezas que se estan evaluando son 3747 ya que esas es la cantidad de beerId únicos en el dataset.

Finalmente la cantidad de ciudadanos que participaron de esta evaluación fueron 706 ya que esa es la cantidad de userId únicos.

## 2.1 Caracterización Inicial [3 puntos]

Antes de entrenar nuestro sistema recomendador, es necesario caracterizar los datos disponibles y de esta manera tener un mejor entendimiento del problema a resolver. Para esto, se le pide lo siguiente:

1. Genere gráficos y/o estadística descriptiva con los datos. Utilice su análisis para responder las siguientes preguntas: [2 puntos]
  - ¿Cuantos *usuarios* hay en los datos?
  - ¿Cuantos *productos* hay en los datos?
  - ¿Cuantas *calificaciones* hay en los datos?
  - ¿Cuantas *calificaciones* faltantes hay en los datos?
  - ¿Cual es la media de las *calificaciones* entregadas por los usuarios? ¿Cómo cambia esto a través de los productos?  
2. Transforme los datos entregados a una matriz (usuario, producto) y muestre sus resultados. [1 punto]

In [ ]:
#1. Generar graficos y/o estadisticas descriptiva con los datos
#cuantos usuarios hya en los datos
print(f"Cuantos usuarios hay en los datos: {df_cervezas['userId'].nunique()}")
#cuantos productos hay en los datos
print(f"Cuantos productos hay en los datos: {df_cervezas['beerId'].nunique()}")
#cuantas calificaciones hya en los datos
print(f"Cuantas calificaciones hay en los datos: {df_cervezas.shape[0]}")
#cuantas calificaciones faltantes hay en los datos
print(f"Cuantas calificaciones faltantes hay en los datos: {df_cervezas['rating'].isna().sum()}")
#media de las calificaicones entregadas por los usuarios
print(f"Media de las calificaciones entregadas por los usuarios:")
df_cervezas.loc[:, ["rating"]].describe().round(4)



In [ ]:
px.histogram(
    df_cervezas,
    x="rating",
    title="Análisis de la variable rating",
    marginal="box"
)

In [ ]:
#2. Transforme los datos entregados a una matriz(usuario, producto) y muestre sus resultados
df_cervezas_pivot = df_cervezas.pivot_table(index='userId', columns='beerId', values='rating', fill_value=0)
df_cervezas_pivot

## 2.2 Método Basado en Contenido [4 puntos]

<center>
<img src ="https://is1-ssl.mzstatic.com/image/thumb/WfNa_TwFpJoTWXQmU-BrbA/1200x675.jpg" width = 500 />

Como primera tarea, Homero le pide centrarse en las características de las cervezas y no en la interacción de los ciudadanos con esta. Adicionalmente, Homero le quiere regalar un pack de distintas cervezas a Carl en agradecimiento a su ayuda anterior, el cual se enteró que Carl tiene un `userId = 100`. Por ende, Homero le pide las siguientes acciones:

1. Cree un dataframe que contenga las características de las cervezas, excluyendo las evaluaciones de los ciudadanos. `Nota:` No considere valores duplicados. [1 punto]
2. Implemente el método Bag of Words para procesar la columna `beerStyle`. Para ello, transforme las palabras a minúsculas, elimine stopwords en inglés y filtre las palabras que aparezcan en 10 documentos como mínimo. Guarde su resultado como un DataFrame. [1 punto].
> `Nota:` Le puede ser útil esta [documentacion](https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) y el método `.get_feature_names_out()`.
3. Cree una función que imprima las top 5 cervezas similares a `Chocolate Porter`, utilizando como medida de similitud la similitud coseno. [1 punto]
4. Identifique la cerveza mejor evaluada por Homero (`userId = 100`) y obtenga las top 5 cervezas similares a esa cerveza. [1 punto]

**Respuesta:**

In [ ]:
#1. Cree un dataframe que contenga las características de las cervezas, excluyendo las evaluaciones de los ciudadanos. Nota: No considere valores duplicados
df_cervezas_caracteristicas = df_cervezas.drop(columns=['rating'])
df_cervezas_caracteristicas = df_cervezas_caracteristicas.drop_duplicates()

In [ ]:
#usuario userId=100
df_cervezas_caracteristicas[df_cervezas_caracteristicas['userId'] == 100]

In [ ]:
#Implemente el método Bag of Words para procesar la columna beerStyle. Para ello, transforme las palabras a minúsculas, elimine stopwords en inglés
#y filtre las palabras que aparezcan en 10 documentos como mínimo. Guarde su resultado como un DataFrame.
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(lowercase = True, # transformación a minúsculas
                             min_df = 10, # mínimo 10 ocurrencias
                             stop_words = 'english') # eliminar stopwords
bow = vectorizer.fit_transform(df_cervezas_caracteristicas['beerStyle']).toarray() # Vectorizamos texto
bow = pd.DataFrame(bow, columns=vectorizer.get_feature_names_out(),index=df_cervezas_caracteristicas['beerId']) # Transformamos a Dataframe
bow

In [ ]:
# importamos cosine_similarity de sklearn
from sklearn.metrics.pairwise import cosine_similarity

# calculamos la similitud coseno de cada vector generado contra el resto
cosine_sim = cosine_similarity(bow, bow)
cosine_sim

In [ ]:
len(cosine_sim)

In [ ]:
#3. Cree una función que imprima las top 5 cervezas similares a Chocolate Porter, utilizando como medida de similitud la similitud coseno.
def get_recommendations(df, cerveza, k = 5):

  """
  Returns the top k similar cervezas using BoW
  """

  # Assert cerveza is contained in dataset
  assert cerveza in df['beerName'].values, 'cerveza is not contained in train dataset!'

  # Get idx of movie
  idx = df[df['beerName'] == cerveza]['beerId'].values[0]
  # print(idx)
  idx_bow = bow.index.get_loc(idx)
  # print(idx_bow)

  # Get pairwise similarities of all cervezas with specified movie
  sim_scores = list(enumerate(cosine_sim[idx_bow]))

  # Sort the cervezas based on the similarity scores
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

  # Get the scores of the k most similar cervezas
  sim_scores = sim_scores[1:k+1]

  # Get the cerveza indices
  cerveza_indices = [i[0] for i in sim_scores]

  # Return the top k most similar movies
  return df['beerName'].iloc[cerveza_indices].values

In [ ]:
top_5=get_recommendations(df = df_cervezas_caracteristicas, cerveza = 'Chocolate Porter')
print(f' top 5 cervezas similares a Chocolate Porter: {top_5}')

In [ ]:
#4. Identifique la cerveza mejor evaluada por Carl (userId = 100) y obtenga las top 5 cervezas similares a esa cerveza.
# primero rescatamos la cerveza más tomada por el usuario
userId = 100 #usuario a recomendar
user_cerveza = df_cervezas[df_cervezas['userId'] == userId].sort_values('rating', ascending = False) # cervezas rateadas por el usuario
best_cerveza = user_cerveza.iloc[0]['beerName'] # cerveza con mejor rating
print(f'most liked cerveza by user {userId}: {best_cerveza}')

In [ ]:
df_cervezas_caracteristicas[df_cervezas_caracteristicas['beerName'] == best_cerveza]

In [ ]:
top_5_userId=get_recommendations(df = df_cervezas_caracteristicas, cerveza = best_cerveza)
print(f'Top 5 de las cervezas más similares a la cerveza most liked for Homero: {top_5_userId}')

## 2.3 Filtros Colaborativos [16 puntos]

<center>
<img src ="https://i.makeagif.com/media/2-08-2016/McmGFQ.gif" width = 400 />

Después de consolidarse como el Barón de la Cerveza en Springfield, Homero Simpson decidió llevar su negocio clandestino a otro nivel. Al ser el único proveedor de alcohol en la ciudad, quiere asegurarse de que cada cerveza que elabora sea de la mejor calidad posible. Para lograrlo, planea usar las evaluaciones pasadas que los ciudadanos de Springfield han registrado sobre las distintas cervezas. Inspirado por los consejos de su amigo Moe, Homero se enteró de que puede implementar un sistema de recomendación basado en filtros colaborativos.

Ahora, Homero necesita de su ayuda para diseñar y elaborar distintos sistemas de filtros colaborativos que le permitan continuar con su reinado cervecero.

### 2.3.1 Filtros Colaborativos Basados en Memoria [8 puntos]

Para este tipo de filtros, Homero le pide lo siguiente:

1. Transforme el DataFrame de `pandas` a Dataset de `surprise`. `Hint`: Utilice solamente los identificadores de ciudadano, cerveza y rating. ¿Importa el orden? [1 punto]
2. Genere un conjunto de entrenamiento y uno de prueba, asegurando que el 30% de los datos sea destinado al conjunto de prueba. No olvide fijar una semilla para garantizar reproducibilidad. [1 punto]
3. Entrene un modelo de **KNN basado en usuarios** utilizando la `similitud coseno`. Luego realice predicciones en el conjunto de prueba. `Nota`: Le puede ser útil la siguiente [documentación](https://surprise.readthedocs.io/en/stable/knn_inspired.html) [2 puntos]
4. Calcule el MAE (Mean Absolute Error) de las predicciones realizadas por el modelo y muestre las predicciones en un DataFrame para su análisis. `Nota:` Le puede ser útil la siguiente [documentación](https://surprise.readthedocs.io/en/stable/accuracy.html#surprise.accuracy.mae) [2 puntos]
5. Repita los pasos 3 y 4 pero utilizando un modelo de **KNN basado en los productos**. [2 puntos]

**Respuesta:**

In [ ]:
df_cervezas['rating'].max()

In [ ]:
#1. trasnformar el dataframe de pandas a dataset de surprise, usar solo identificadores de ciudadano, cerveza y rating
reader = Reader(rating_scale=(0, 5))
df_surprise = Dataset.load_from_df(df_cervezas[['userId',  'beerId','rating']], reader)

*1. Importa el orden?*

Si, el orden en esta ocasión importa ya que define cual variable se usará como usuario, item y valoración en el dataset y en ese mismo orden, es por esto que se define el orden de userId, beerId y rating.

In [ ]:
#2. Genere un conjunto de entrenamiento y uno de prueba,
#asegurando que el 30% de los datos sea destinado al conjunto de prueba.
#No olvide fijar una semilla para garantizar reproducibilidad.
from surprise.model_selection import train_test_split
trainset, testset = train_test_split(df_surprise, test_size=0.3, random_state=3380)

In [ ]:
#3.Entrene un modelo de KNN basado en usuarios utilizando la similitud coseno. Luego realice predicciones en el conjunto de prueba
from surprise.similarities import cosine
model_knn=KNNBasic(sim_options= {'name': 'cosine', 'user_based': True})
model_knn.fit(trainset)
predictions=model_knn.test(testset)
df_predictions = pd.DataFrame(predictions)
df_predictions

In [ ]:
#4. Calcule el MAE (Mean Absolute Error) de las predicciones
#realizadas por el modelo y muestre las predicciones en un DataFrame
#para su análisis
from surprise.accuracy import mae
mae(predictions)

In [ ]:
#5. Repita los pasos 3 y 4 pero utilizando un modelo de KNN
#basado en los productos
model_knn_products=KNNBasic(sim_options={'name': 'cosine', 'user_based': False})
model_knn_products.fit(trainset)
predictions_products=model_knn_products.test(testset)
df_predictions_products = pd.DataFrame(predictions_products)
df_predictions_products

In [ ]:
mae(predictions_products)

### 2.3.2 Filtros Colaborativos Basados en Modelo [8 puntos]

Ahora Homero le pide que cambie el enfoque del tipo de filtro colaborativo a uno basado en modelo. Por ende, Homero le pide lo siguiente:

1. Entrene un modelo `NMF` , asegurándose de fijar la semilla para reproducibilidad. Luego genere predicciones para el conjunto de prueba. [2 puntos]
2. Calcule el MAE de las predicciones obtenidas y muestre los resultados de las predicciones en un DataFrame para su analisis. [1 punto]
3. Entrene un modelo `SVD` sobre el conjunto de entrenamiento. Posteriormente, genere las predicciones para el conjunto de prueba. [2 puntos]
4. Calcule el MAE de las predicciones y muestre los resultados de las predicciones en un DataFrame. [1 punto]
5. Compare los resultados de ambos modelos y seleccione el mejor a su criterio. Justifique. [2 puntos]

**Respuesta**

In [ ]:
#1. entrene modelo NMF, asegure semilla para reproducibilidad. Genere predicciones conjunto prueba
model_nmf = NMF(random_state=3380)
model_nmf.fit(trainset)
predictions_nmf = model_nmf.test(testset)
#2. calcule MAE de las predis obtenidas y muestre results en un dataframe
mae(predictions_nmf)
df_predictions_nmf = pd.DataFrame(predictions_nmf)
df_predictions_nmf

In [ ]:
#3.entrena modelo SVD sobre el conjunto entrenamiento. dsps genere predis en el conjunto de prueba
model_svd = SVD()
model_svd.fit(trainset)
predictions_svd = model_svd.test(testset)
#4. calcula mae y lleve a dataframe los results
mae(predictions_svd)
df_predictions_svd = pd.DataFrame(predictions_svd)
df_predictions_svd

*5. Comparación de modelos*

De los modelos vistos anteriormente se obtienen MAE's similares entre 0.46 a 0.48, estos son valores bajos indicando que las predicciones del modelo están muy cercanas al valor real. Además es esperable un valor bajo ya que la variable que se está poniendo en juego es rating, la cual no tiene una magnitud grande ya que solo se mueve entre 0 y 5, en cambio si fuera un rango mayor, probablemente el mae aumentaría.

Por lo anterior, se escoge como mejor modelo el que posee menor error, es decir menor MAE, el cual es el modelo SVD con un mae=0.4627

## 2.4 El Mejor Filtro Colaborativo [7 puntos]

<center>
<img src ="https://media.giphy.com/media/citBl9yPwnUOs/giphy.gif" width = 400 />

Homero Simpson desea perfeccionar y extender su sistema de recomendaciones para cervezas en Springfield. En esta fase, busca implementar técnicas que le permitan no solo predecir las preferencias de los ciudadanos, sino también evaluar métricas que midan la eficacia de sus recomendaciones. Con el objetivo de recomendar las mejores K cervezas para usuarios específicos, Homero buscará comprender mejor cómo optimizar su sistema y satisfacer a sus clientes.

Para ello, usted ayudará a Homero realizando las siguientes labores:

1. Implemente la función `get_top_n()` que le permita extraer las `N` mejores cervezas recomendadas para cada ciudadano utilizando las predicciones generadas por el mejor modelo seleccionado en la sección anterior. Luego obtenga las `3` mejores cervezas recomendadas para Homero (`userId = 100`). `Hint`: Para cada ciudadano, elimine las predicciones duplicadas. [2 puntos]
2. Realice predicciones de rating para 5 cervezas aleatorias no evaluadas por Homero, para estimar las calificaciones que Homero podría darles. [1 punto]
3. Calcule los promedios de `precisión@k` y el `recall@k` para `k=10` y `threshold = 3.5`. Analice sus resultados. ¿Es un buen modelo? `Hint:` Utilice la función dada `precision_recall_at_k(...)`. [1 punto]
4. Calcule la `precisión@k` y el `recall@k` para valores de `k` que varíen entre 3 y 25 con paso de 1 unidad. Luego, grafique los resultados y analice el comportamiento de las curvas de precisión y recall en función de `k`. [1 punto]
5. Calcule la `precisión@k` y el `recall@k` para valores de `threshold` que varíen entre 3.0 y 4.5, con paso de 0.1. Luego, grafique los resultados y analice el comportamiento de las curvas de precisión y recall en función de `threshold`. [1 punto]
6. Finalmente utilice el mejor modelo para estimar los ratings faltantes. Muestre sus resultados en una matriz (usuario, producto) sin valores nulos. [1 punto]



In [ ]:
#NO MODIFICAR ESTA CELDA
def precision_recall_at_k(predictions, k=10, threshold=3.5):

    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))
    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])
        n_rel_and_rec_k = sum(
            ((true_r >= threshold) and (est >= threshold))
            for (est, true_r) in user_ratings[:k]
        )
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0
    return precisions, recalls

**Respuesta:**

In [ ]:
#1.implementar get_top_n que permita extraer n mejores cervezas recomendadas para cada ciudadano usando predis por el mejor modelo seleccionado anterior
#luego las mejores 3 cervezas recomendadas para homero user=100. Elimine las predis duplicadas
from collections import defaultdict

def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        #duplicados de predis
        if est not in [x[1] for x in top_n[uid]]:
            top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n



In [ ]:
#obtengo las de homero de get top n
homero_top_n = get_top_n(predictions_svd, n = 3)
homero_top_n.get(100, [])

In [ ]:
#2Realice predicciones de rating para 5 cervezas aleatorias no evaluadas por Homero, para estimar las calificaciones que Homero podría darles
import random

#Lista de todas las cervezas
all_beers = set(df_cervezas['beerId'])

#Obtener las cervezas que homero a evaluado
evaluadas_por_homero = list(df_cervezas[df_cervezas['userId'] == 100]['beerId'])

#Obtener cervezas no evaluadas por homero
no_evaluadas_por_homero = list(all_beers - set(evaluadas_por_homero))

#5 cervezas aleatorias
random_beers = random.sample(no_evaluadas_por_homero, 5)

#Predecir el rating para las 5 cervezas seleccionadas
pre_random=[]
for beer in random_beers:
    pre_random.append( model_nmf.predict(100, beer))

predi_random = pd.DataFrame(pre_random)
predi_random


In [ ]:
#3 Calcule los promedios de precisión@k y el recall@k para k=10 y threshold = 3.5.
# Analice sus resultados. ¿Es un buen modelo? Hint: Utilice la función dada precision_recall_at_k(...)
precisions, recalls = precision_recall_at_k(predictions_nmf, k=10, threshold=3.5)
precision = np.mean([precision for precision in precisions.values()])
recall = np.mean([recall for recall in recalls.values()])
print(f"Precisión@k: {precision:.2f}, Recall@k: {recall:.2f}")

*3. es un buen modelo?*

Un bajo recall nos indica que el modelo capturó pocos casos positivos correctamente, se suele buscar un modelo con recall alto para evitar costos asociados a un falso negativo. Por otro lado en precisión tiene una alta tasa de predicción correcta, así se concluye que el modelo no es bueno ya que posee bajo recall y la idea es tener el mayor recall posible a pesar de que tenga una buena precision.

In [ ]:
#4. Calcule la precisión@k y el recall@k para valores de k que varíen entre 3 y 25 con paso de 1 unidad.
#Luego, grafique los resultados y analice el comportamiento de las curvas de precisión y recall en función de k
r=list(range(3,26))
pre=[]
recall=[]
for i in r:
  precisions, recalls = precision_recall_at_k(predictions_nmf, k=i, threshold=3.5)
  mean_precision=np.mean([precision for precision in precisions.values()])
  mean_recall=np.mean([recall for recall in recalls.values()])
  pre.append(mean_precision)
  recall.append(mean_recall)


In [ ]:
#gafico resultados de pre y recall con plotly
#grafico precision
fig = px.line(x=r, y=pre, title='Gráfico Precisión@k')
fig.show()
#grafico recall
fig = px.line(x=r, y=recall, title='Gráfico Recall@k')
fig.show()
#grafio PR
fig = px.line(x=recall, y=pre, title='Grpafico PR')
fig.show()

*Comentario*

En lo gráficos se puede observar que a medida que el k aumenta tambien lo hace el recall, en cambio el precision baja. Lo anterior se puede deber a que varia la cantidad de datos.

Además se agrega el gráfico PR el cual muestr una curva similar a la curva ideal de referencia que deben tener los gráficos Precision/recall

In [ ]:
#5. Calcule la precisión@k y el recall@k para valores de threshold que varíen entre 3.0 y 4.5, con paso de 0.1.
# Luego, grafique los resultados y analice el comportamiento de las curvas de precisión y recall en función de threshold
new_step=[3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4.0, 4.1, 4.2, 4.3, 4.4, 4.5]
pre_2=[]
recall_2=[]
for i in new_step:
  precisions, recalls = precision_recall_at_k(predictions_nmf, k=10, threshold=i)
  mean_precision=np.mean([precision for precision in precisions.values()])
  mean_recall=np.mean([recall for recall in recalls.values()])
  pre_2.append(mean_precision)
  recall_2.append(mean_recall)

In [ ]:
#grafico precision
fig = px.line(x=new_step, y=pre_2, title='Gráfico Precisión@k')
fig.show()
#grafico recall
fig = px.line(x=new_step, y=recall_2, title='Gráfico Recall@k')
fig.show()
#grafio PR
fig = px.line(x=recall_2, y=pre_2, title='Gráfico PR')
fig.show()

*Comentario*


A medida que el threshold aumenta, el precision cae, pero con un cambio constante y no de manera abrupta. Por otro lado, el recall tambien disminuye lentamente.

Adicionalmente se grafica la curva Precision/Recall la cual está alejada de ser una curva ideal

In [ ]:
#6Finalmente utilice el mejor modelo para estimar los ratings faltantes. Muestre sus resultados en una matriz (usuario, producto) sin valores nulos.

# Get missing ratings, es decir los que tienen valor 0
df_cervezas_pivot.replace(0, np.nan, inplace=True)
missing_ratings = df_cervezas_pivot.isna()

#recorro indice de userId y luego de beerId
for userId in df_cervezas_pivot.index:
  for beerId in df_cervezas_pivot.columns:
    #si está NaN que realice la predi
    if missing_ratings.loc[userId, beerId]:
      df_cervezas_pivot.loc[userId, beerId] = model_nmf.predict(userId, beerId).est


In [ ]:
df_cervezas_pivot

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por el foro de U-cursos o por correo.

<center>
<img src ="https://i.gifer.com/origin/cb/cbd80e6045652ab123caffef72f29210_w200.gif" width = 400 />
